In [1]:
import selenium
from selenium import webdriver
import time
import requests
from PIL import Image
import io
from io import BytesIO
import os
import hashlib


In [2]:
DRIVER_PATH = "/Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/chromedriver"
wd = webdriver.Chrome(executable_path= DRIVER_PATH)

/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_67048/708202962.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path= DRIVER_PATH)


In [3]:
wd.get('https://google.com')



In [4]:
# search_box = wd.find_element_by_css_selector('input.gLFyf')
# search_box.send_keys('Dogs')

In [5]:
wd.quit()

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return 
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [7]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

Persist all images

In [8]:

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

Searching download and saving images

In [9]:
search_term_cutlery = "plastic cutlery"
search_term_cups = "plastic cup"
search_term_bottle = "plastic bottle"
search_term_cutlery_dirty = "dirty plastic cutlery"
search_term_cups_dirty = " dirty plastic cup"
search_term_bottle_dirty = " dirty plastic bottle"
driver_path = DRIVER_PATH
target_path = "/Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images"
number_images = 40
search_and_download(search_term_bottle, driver_path, target_path, number_images)
search_and_download(search_term_cups, driver_path, target_path, number_images)
search_and_download(search_term_cutlery, driver_path, target_path, number_images)

search_and_download(search_term_bottle_dirty, driver_path, target_path, number_images)
search_and_download(search_term_cups_dirty, driver_path, target_path, number_images)
search_and_download(search_term_cutlery_dirty, driver_path, target_path, number_images)

/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_67048/2975638876.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_67048/2109922686.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_67048/2109922686.py:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 41 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSJK2EqykspRcrj70l3bn-0xYqt_pymoO1ssA&usqp=CAU - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/6874b1a292.jpg
SUCCESS - saved https://st2.depositphotos.com/5118587/7519/i/950/depositphotos_75192165-stock-photo-crushed-plastic-bottle.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/ab8001822d.jpg
SUCCESS - saved https://tiimg.tistatic.com/fp/1/007/028/pure-water-plastic-bottle-272.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/810c4de7cf.jpg
SUCCESS - saved https://ideon.co.uk/wp-content/uploads/2018/06/1000mljuice.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/65850d0ac4.jpg
SUCCESS - saved https://media.istockphoto.com/vectors/plastic-bottle-with-mineral-water-on-alpha-transp

/Users/enyonadjanor/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://images.twinkl.co.uk/tr/image/upload/t_illustration/illustation/plastic-bottle.png - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/9f4c180665.jpg
SUCCESS - saved https://media.istockphoto.com/photos/plastic-bottle-crushed-for-recycling-on-white-background-picture-id1087277208 - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/d6590c5b75.jpg
SUCCESS - saved https://media.healthyfood.com/wp-content/uploads/2017/03/Fact-or-fiction-Freezing-plastic.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/ea1c74673d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3cbEqzFpbaU3-pmqnP2lGR9ZZz7yVUK1m2A&usqp=CAU - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/eb72473e7c.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/empty-plastic-bottle-isolated-whi